In [1]:
# iMPORT PACKAGES
import pandas as pd
import datetime as dt
import numpy as np

from datetime import timedelta

In [ ]:
# READ FILE
inputFile = "dataset_mood_smartphone.csv"
df = pd.read_csv(inputFile, sep = ',')

In [ ]:
# DATE TO DATETIME 
def getDate(x):
    return x.date()
    
df['date'] = pd.to_datetime(df['time'])
df['date'] = df['date'].apply(getDate)

In [ ]:
print(min(df['date']))
print(max(df['date']))

print((max(df['date'])-min(df['date'])).days)

In [ ]:
ind = df[(df['value']<0) & (df['variable'] != 'circumplex.arousal') & (df['variable'] != 'circumplex.valence')].index
df = df.drop(df.index[ind])

In [ ]:
df.head()

In [ ]:
# MAKE PIVOT

dfGrouped = df.groupby(['id', 'date', 'variable'])['value'].agg(['mean', 'sum']).reset_index()
dfPivot = dfGrouped.pivot_table(index=['id','date'], columns='variable', values=['mean','sum'])

# only keep relevant variables
dfPivot = dfPivot[[('mean', 'activity'), ('mean', 'circumplex.arousal'), ('mean', 'circumplex.valence'), 
         ('mean', 'mood'), ('sum', 'appCat.builtin'), ('sum', 'appCat.communication'), 
         ('sum', 'appCat.entertainment'), ('sum', 'appCat.finance'), ('sum', 'appCat.game'), 
        ('sum', 'appCat.office'), ('sum', 'appCat.other'), ('sum', 'appCat.social'), ('sum', 'appCat.travel'),
        ('sum', 'appCat.unknown'), ('sum', 'appCat.utilities'), ('sum', 'appCat.weather'), ('sum', 'call'), 
         ('sum', 'screen'), ('sum', 'sms')]].reset_index()

# rename columns
cols = ['id', 'date','activity_mean', 'circumplex.arousal_mean',
       'circumplex.valence_mean', 'mood_mean', 'appCat.builtin_sum',
       'appCat.communication_sum', 'appCat.entertainment_sum',
       'appCat.finance_sum', 'appCat.game_sum', 'appCat.office_sum',
       'appCat.other_sum', 'appCat.social_sum', 'appCat.travel_sum',
       'appCat.unknown_sum', 'appCat.utilities_sum', 'appCat.weather_sum',
       'call_sum', 'screen_sum', 'sms_sum']
dfPivot.columns = cols

print(len(dfPivot))
dfPivot.head()

In [ ]:
# ADD VALUES DAY AND WEEKDAY
df = dfPivot

def getDay(x):
    return x.days

def getWeekday(x):
    return x.weekday()

df['day'] = (df['date']-min(df['date']))
df['day'] = df['day'].apply(getDay)
df['weekDay'] = df['date'].apply(getWeekday)

df.head()

In [ ]:
# FILL NAN VALUES OF CERTAIN COLUMNS WITH 0 
df[['activity_mean','circumplex.arousal_mean', 'circumplex.valence_mean', 'appCat.builtin_sum',
       'appCat.communication_sum', 'appCat.entertainment_sum',
       'appCat.finance_sum', 'appCat.game_sum', 'appCat.office_sum',
       'appCat.other_sum', 'appCat.social_sum', 'appCat.travel_sum',
       'appCat.unknown_sum', 'appCat.utilities_sum', 'appCat.weather_sum',
       'call_sum', 'screen_sum', 'sms_sum']] = df[['activity_mean', 'circumplex.arousal_mean', 
        'circumplex.valence_mean', 'appCat.builtin_sum',
       'appCat.communication_sum', 'appCat.entertainment_sum',
       'appCat.finance_sum', 'appCat.game_sum', 'appCat.office_sum',
       'appCat.other_sum', 'appCat.social_sum', 'appCat.travel_sum',
       'appCat.unknown_sum', 'appCat.utilities_sum', 'appCat.weather_sum',
       'call_sum', 'screen_sum', 'sms_sum']].fillna(0)

df.head()

In [ ]:
print(df['mood_mean'].isna().sum())
print(len(df))

In [ ]:
""" 
#df.to_csv('out.csv', index = False)

df = pd.read_csv('out.csv', sep = ',')
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].apply(getDate)
df.head()
"""

In [ ]:
## ADD TIME COLUMNS

timePeriods = 5 
colNames = df.columns.values

for timePeriod in range(1,timePeriods+1):
    for colname in colNames:
        if (colname != "id") and (colname != "date"):
            df[colname+"_time_"+str(timePeriod)] = df.groupby('id')[colname].rolling(window=timePeriod, min_periods = 1).mean().reset_index().iloc[:,2]
        
print(len(df))

In [ ]:
# ADD (REAL) TARGET VALUE 

df['previousDay'] = df['date'] + timedelta(days = -1)

target = df[['id','previousDay', 'mood_mean']]
target.columns = ['id','date', 'mood_mean_TARGET']

df = df.set_index(['id','date']).join(target.set_index(['id','date'])).reset_index()

df = df.drop('previousDay', axis = 1)

print(len(df))

In [ ]:
for j in range (1,10):
    curColumn = []
    curColumn5DaysMood = []
    normalCounter = 0
    curID = ""
    counterCurJMood = 0
    counterJmoodLast5Days = 0
    for i in range(len(df[['id','mood_mean']])):
        curRow = (df[['id','mood_mean']].iloc[i])
        if normalCounter == 5:
            counterJmoodLast5Days = 0
        
        if curID != curRow['id']:
            curID= curRow['id']
            normalCounter=0
            counterJmoodLast5Days=0
            counterCurJMood = 0
        if (float(curRow['mood_mean']) >= j and float(curRow['mood_mean']) < j+1):
            counterJmoodLast5Days += 1
        if (float(curRow['mood_mean']) >= j):
            counterCurJMood = 0
        else:
            counterCurJMood += 1
        
        curColumn.append(counterCurJMood)
        curColumn5DaysMood.append(counterJmoodLast5Days)
    
    df['lastTimeSeenMood_'+str(j)] = curColumn
    df['numberOfTimesSeenMood5Days_'+str(j)] = curColumn5DaysMood

In [ ]:
""" 
## NORMALISE list
def normaliseDF(df):
    maxList = []
    minList = []
    for column in df.columns:
        maxi = df[column].max()
        maxList += [maxi]
        mini = df[column].min()
        minList += [mini]
        if (column != "id") and (column != "date"): # and (column != "mood_mean"):
            df[column] = ((df[column]-mini)/(maxi-mini))
    return df, maxList, minList

df, maxList, minList = normaliseDF(df)
"""

In [ ]:
## NORMALISE dict
def normaliseDF(df):
    maxDict = {}
    minDict = {}
    for column in df.columns:
        maxi = df[column].max()
        maxDict[column] = maxi
        mini = df[column].min()
        minDict[column] = mini
        if (column != "id") and (column != "date") and (column != "mood_mean_TARGET"):
            df[column] = ((df[column]-mini)/(maxi-mini))
    return df, maxDict, minDict

df, maxDict, minDict = normaliseDF(df)

In [ ]:
## REVERSE - NORMALISE 
def reverseNormaliseDF(df, maxDict, minDict):
    for column in df.columns:
        maxi = maxDict[column]
        mini = minDict[column]
        if (column != "id") and (column != "date") and (column != "mood_mean"):
            df[column] = (df[column]*(maxi-mini)) + mini
    return df

#df = reverseNormaliseDF(df, maxDict, minDict)

In [ ]:
#df.to_csv("out_with_nan_mood_normalised.csv", index = False)

In [ ]:
#df.dropna(subset = ['mood_mean'] ).to_csv("out_without_nan_mood_normalised.csv", index = False)

In [ ]:
#df.dropna(subset = ['mood_mean_TARGET'] ).to_csv("out_without_nan_mood_target_normalised.csv", index = False)

In [22]:
dfNonNan = df.dropna(subset = ['mood_mean_TARGET'] )

In [24]:
# SHOW CORRELATIONS
corr = dfNonNan.corr()
corr[['mood_mean_TARGET','mood_mean']].style.background_gradient(cmap='coolwarm')

,mood_mean_TARGET,mood_mean
activity_mean,0.112584,0.150103
circumplex.arousal_mean,0.0808079,0.157639
circumplex.valence_mean,0.250137,0.687242
mood_mean,0.472047,1
appCat.builtin_sum,0.011765,-0.0415037
appCat.communication_sum,0.0549667,0.0478215
appCat.entertainment_sum,0.104142,0.0902602
appCat.finance_sum,0.0139657,0.0198443
appCat.game_sum,0.0483083,0.0727129
appCat.office_sum,0.0588372,0.0399645


In [25]:
corr[abs(corr['mood_mean_TARGET'])>0.1]['mood_mean_TARGET'].sort_values()

weekDay_time_5                    -0.107756
appCat.entertainment_sum_time_1    0.104142
appCat.entertainment_sum           0.104142
appCat.other_sum_time_4            0.105976
appCat.game_sum_time_3             0.108747
activity_mean_time_3               0.109724
appCat.entertainment_sum_time_3    0.112453
activity_mean_time_1               0.112584
activity_mean                      0.112584
activity_mean_time_2               0.119888
appCat.other_sum_time_5            0.121189
appCat.game_sum_time_4             0.122164
appCat.game_sum_time_5             0.123017
appCat.entertainment_sum_time_4    0.123766
appCat.entertainment_sum_time_5    0.130096
circumplex.valence_mean_time_5     0.220753
circumplex.valence_mean_time_3     0.223433
circumplex.valence_mean_time_4     0.224724
circumplex.valence_mean_time_1     0.250137
circumplex.valence_mean            0.250137
circumplex.valence_mean_time_2     0.251291
mood_mean                          0.472047
mood_mean_time_1                

In [ ]:
# MAKE TEST AND TRAIN SET
df = df.dropna(subset = ['mood_mean','mood_mean_TARGET'] ).reset_index(drop=True)
dfTest = df.groupby("id").tail(12).reset_index(drop =True)
dfTrain = (df.ix[df.index.difference(df.groupby('id').tail(12).index)])

dfTest.to_csv("test.csv", index = False)
dfTrain.to_csv("train.csv", index = False)